In [1]:
import pandas as pd
df = pd.read_csv("~/documents/Inputs1stGen.csv")

In [2]:
print(df)

                      by   karma  \
0            ColinWright  127765   
1                   hncj      44   
2             andymboyle     714   
3                digisth    2395   
4                jazzdev     543   
...                  ...     ...   
5116176          d4vinci       1   
5116177  giuliomagnifico   40731   
5116178    retronick2020      78   
5116179   ElizabethMoore       1   
5116180          partime       0   

                                                     title  \
0        "What May Happen in the Next Hundred Years", f...   
1             Getting Started with JavaScript Unit Testing   
2        Armstrong, the Django-based and open-source ne...   
3                     Why Web Reviewers Make Up Bad Things   
4        You Weren't Meant to Have a Boss: The Cliff Notes   
...                                                    ...   
5116176  Scrapling: Fast, Adaptive Web Scraping for Python   
5116177                       Monkeys Predict US Elections   
5116178  

In [3]:
from datetime import datetime
from urllib.parse import urlparse

'''
The following section gives us all the parameters relating to time:
1. Day of week
2. Time of day
3. Month of Year
4. Year of past 
'''

#Converting UNIX timestamp to datetime
df['datetime'] = pd.to_datetime(df['time'])

#Getting the names of the days of the week
df['day_of_week'] = df['datetime'].dt.day_name()

#Creating a key of days of the week to numbers
day_map = {
    'Monday': 0, 'Tuesday': 1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6
}

#Assigning numbers to the days of the week corresponding to the dataset
df["day_of_week_num"] = df['day_of_week'].map(day_map)

#Creating a key of months of the year to numbers
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April':4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October':10, 'November': 11, 'December': 12
}

#df['month_num'] = df['month'].map(month_map)

#Getting the hour of the post
df['hour'] = df['datetime'].dt.hour

#Getting the year of the post
df['year'] = df['datetime'].dt.year


In [4]:
print(df)

                      by   karma  \
0            ColinWright  127765   
1                   hncj      44   
2             andymboyle     714   
3                digisth    2395   
4                jazzdev     543   
...                  ...     ...   
5116176          d4vinci       1   
5116177  giuliomagnifico   40731   
5116178    retronick2020      78   
5116179   ElizabethMoore       1   
5116180          partime       0   

                                                     title  \
0        "What May Happen in the Next Hundred Years", f...   
1             Getting Started with JavaScript Unit Testing   
2        Armstrong, the Django-based and open-source ne...   
3                     Why Web Reviewers Make Up Bad Things   
4        You Weren't Meant to Have a Boss: The Cliff Notes   
...                                                    ...   
5116176  Scrapling: Fast, Adaptive Web Scraping for Python   
5116177                       Monkeys Predict US Elections   
5116178  

In [5]:
max_time = df['datetime'].max()
print(max_time)

2024-10-14 00:00:50


In [6]:
df['time_since_post1'] = max_time - df['datetime']

In [7]:
print('time_since_post1')

time_since_post1


In [8]:
print(df['time_since_post1'])

0         4738 days 07:33:50
1         4647 days 12:21:25
2         4738 days 07:33:14
3         4107 days 18:44:24
4         6041 days 14:14:25
                 ...        
5116176      0 days 00:11:08
5116177      0 days 00:07:50
5116178      0 days 00:03:39
5116179      0 days 00:02:32
5116180      0 days 00:00:00
Name: time_since_post1, Length: 5116181, dtype: timedelta64[ns]


In [9]:
'''
The following code gives us:
2. Time since the post was posted
'''

df['time_since_post'] = df['time_since_post1'].dt.total_seconds()

In [10]:
print(df['time_since_post'])

0          409390430.0
1          401545285.0
2          409390394.0
3          354912264.0
4          521993665.0
              ...     
5116176          668.0
5116177          470.0
5116178          219.0
5116179          152.0
5116180            0.0
Name: time_since_post, Length: 5116181, dtype: float64


In [ ]:
'''
The following code gives us:
1. URL
2. Domain
'''

# Define a safe parser
def safe_urlparse(url):
    try:
        parsed = urlparse(url)
        return parsed.netloc, parsed.path
    except Exception:
        return '', ''

# Apply safely
df['url'] = df['url'].fillna('').astype(str)  # Ensure it's string
df[['domain_name', 'url_path']] = df['url'].apply(
    lambda u: pd.Series(safe_urlparse(u))
)

In [ ]:
urlparse('')

In [ ]:
print(df['url'])

In [ ]:
'''
The following code gives us:
1. User name
2. Title
3. Length of Title

'''


